In [3]:
import pandas as pd
import seaborn as sns
import itertools
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats
from statsmodels.formula.api import ols, glm

In [4]:
df_1bed_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_1_bed_flat_monetary_2011-2021.csv')
bed_2_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_flat_monetary_2011-2021.csv')
bed_2_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_2_house_monetary_2011-2021.csv')
bed_3_flat=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_flat_monetary_2011-2021.csv')
bed_3_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_3_house_monetary_2011-2021.csv')
bed_4_house=pd.read_csv('../data/curated/Rental_Price_historical/joined_bed_4_house_monetary_2011-2021.csv')

In [5]:
def impute_missing(df):
    for col in df:
        df[col] = df[col].replace("-","-1")
        if "Median" in col:
            df[col] = df[col].replace({'\$':''}, regex = True)
            df[col] = df[col].replace(",","", regex=True)
            df[col] = pd.to_numeric(df[col])
    return df

In [6]:
df_1bed_flat = impute_missing(df_1bed_flat)
bed_2_flat = impute_missing(bed_2_flat)
bed_2_house = impute_missing(bed_2_house)
bed_3_flat = impute_missing(bed_3_flat)
bed_3_house = impute_missing(bed_3_house)
bed_4_house = impute_missing(bed_4_house)

In [12]:
bed_4_house

,Suburb,Count_Mar2011,Median_Mar2011,Count_Jun2011,Median_Jun2011,Count_Sep2011,Median_Sep2011,Count_Dec2011,Median_Dec2011,Count_Mar2012,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb.1,Postcode.1
0,Albert Park-Middle Park-West St Kilda,19,950,19,860,30,1100,34,1100,37,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
1,Armadale,-1,-1,-1,-1,12,823,16,1100,15,...,2.640000,2.570000,2.8,2.430000,2.360000,2.290000,2.220000,2.0,Lilydale,3143
2,Carlton North,20,745,21,770,17,770,20,760,21,...,1.360000,1.330000,0.0,1.270000,1.240000,1.210000,1.180000,1.8,"Hotham Hill, North Melbourne",3053
3,Carlton-Parkville,37,775,36,766,35,750,31,750,30,...,2.706667,2.636667,2.8,2.496667,2.426667,2.356667,2.286667,2.1,"Attwood, Westmeadows",3052
4,CBD-St Kilda Rd,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Torquay,121,440,120,450,126,450,143,450,160,...,2.560000,2.580000,2.6,2.620000,2.640000,2.660000,2.680000,2.7,Ocean Grove,3228
140,Traralgon,140,330,146,330,153,335,156,340,143,...,2.233333,2.233333,1.9,2.233333,2.233333,2.233333,2.233333,2.4,Nicholson,3844
141,Warragul,54,320,64,330,76,330,72,340,78,...,2.546667,2.556667,2.8,2.576667,2.586667,2.596667,2.606667,2.5,"Valencia Creek, Briagolong, Coongulla, Maffra,...",3820
142,Warrnambool,129,380,124,378,129,380,139,380,129,...,2.773333,2.703333,2.6,2.563333,2.493333,2.423333,2.353333,2.3,"Bushfield, Grassmere, Winslow, Woodford",3280


In [97]:
CORR_COLS2 = ['Median_Mar', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 
             'Median_rent_weekly', 'Median_tot_fam_inc_weekly','Average_num_psns_per_bedroom', 
             'Average_household_size']

yrs = [2011,2012,2013,2014,2015,2016,2017,2018,2019, 2020, 2021]

In [8]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [99]:
# def get_forecast_all_suburbs(raw_df):
#     forecasted_data = {}
#     for index, row in raw_df.iterrows():
#         X_df = make_2D(row)
# #         reg_forecast = forecast(X_df)
#         forecasted_data[row['Suburb']] = forecast(X_df)
#     return forecasted_data

def get_forecast_all_suburbs(raw_df):
    # cols_imputed = {}
    forecasted_data = {}
    for index, row in raw_df.iterrows():
        X_df = make_2D(row)
#             print(X_df)
#         for col in X_df:
#             if col != 'year':
#                 reg = LinearRegression().fit(X_df['year'].values.reshape((-1,1)), X_df[col].values.reshape((-1,1)))
#                 arr = []
#                 for yr in yrs:
#                     arr.extend(reg.predict(np.array([[yr]])))
# #                     print(arr)
#             cols_imputed[col] = arr

#         for i in range(0,len(yrs)):
#             for col in X_df:
#                 if (X_df.loc[i,col] == -1):
#                     X_df.loc[i,col] = cols_imputed[col][i]


        forecasted_data[row['Suburb']] = forecast(X_df)


    return forecasted_data

def make_2D(suburb_row):
#     suburb_row = df_1bed_flat.iloc[0,:]
    cols_imputed = {}
    X_df = {}
    for feature in CORR_COLS2:
        f = []
        for t in suburb_row.iteritems():
            if feature in t[0]:
                f.append(t[1])
        X_df[feature] = f

    X_df = pd.DataFrame.from_dict(X_df)
    X_df.loc[:,'year'] = yrs
    X_df.rename(columns = {"Median_Mar":"Median_rental_price"}, inplace = True)
    
    for col in X_df:
        year = []
        data = []
        if col != 'year':
            for i in range(0,len(yrs)):
                if (X_df.loc[i,col] != -1):
                    data.append(X_df.loc[i,col])
                    year.append(yrs[i]) 
            if (len(year) == 0):
                continue   
            df = pd.DataFrame()     
            df["year"]=year
            df["data"]=data 
            reg = LinearRegression().fit(df[["year"]], df["data"])
            
            arr = []
            for yr in yrs:
                arr.extend(reg.predict(np.array([[yr]])))
    #                     print(arr)
        cols_imputed[col] = arr


    for i in range(0,len(yrs)):
        for col in cols_imputed:
            if (X_df.loc[i,col] == -1):
                X_df.loc[i,col] = cols_imputed[col][i]

    return X_df


def forecast(X_df):
    future_yrs = [2022,2023,2024,2025]
    reg_forecast = pd.DataFrame()
    for col in X_df.iloc[:, 1:8]:
        l = []
        reg = LinearRegression().fit(X_df['year'].values.reshape((-1,1)), X_df[col].values.reshape((-1,1)))
        for yr in future_yrs:
            l.extend(reg.predict(np.array([[yr]])))
            row = [np.round(x[0], 1) for x in l]
    #         print(l)
        reg_forecast[col] = row
    reg_forecast['year'] = future_yrs

    regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    regr.fit(X_df.iloc[:,[1,2,3,7]], X_df['Median_rental_price'])
    future_price = regr.predict(reg_forecast.iloc[:,[0,1,2,6]])
    reg_forecast["Median_rental_price"] = future_price
    # reg = LinearRegression().fit(X_df.iloc[:,[1,2,3,7]], X_df['Median_rental_price'])

    # future_price = reg.predict(reg_forecast.iloc[:,[0,1,2,6]])
    # reg_forecast["Median_rental_price"] = future_price
    return reg_forecast

In [83]:
def make_2D_2(suburb_row):
#     suburb_row = df_1bed_flat.iloc[0,:]
    X_df = {}
    for feature in CORR_COLS2:
        f = []
        for t in suburb_row.iteritems():
            if feature in t[0]:
                f.append(t[1])
        X_df[feature] = f

    X_df = pd.DataFrame.from_dict(X_df)
    X_df.loc[:,'year'] = yrs
    X_df.rename(columns = {"Median_Mar":"Median_rental_price"}, inplace = True)
    
    return X_df

In [100]:
test = get_forecast_all_suburbs(bed_4_house)

/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/ohanr/.local/lib/python3.10/site-packages/sklearn/base.py:4

In [96]:
print(bed_4_house.loc[2, "Suburb"])
make_2D(bed_4_house.iloc[2,:])

Carlton North


ValueError: Length of values (9) does not match length of index (11)

In [11]:
bed_2_flat

,Suburb,Count_Mar2011,Median_Mar2011,Count_Jun2011,Median_Jun2011,Count_Sep2011,Median_Sep2011,Count_Dec2011,Median_Dec2011,Count_Mar2012,...,Average_household_size_2014,Average_household_size_2015,Average_household_size_2016,Average_household_size_2017,Average_household_size_2018,Average_household_size_2019,Average_household_size_2020,Average_household_size_2021,Suburb.1,Postcode.1
0,Albert Park-Middle Park-West St Kilda,201,430,197,420,191,430,211,425,235,...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
1,Armadale,286,380,290,383,277,390,269,395,265,...,2.640000,2.570000,2.8,2.430000,2.360000,2.290000,2.220000,2.0,Lilydale,3143
2,Carlton North,106,400,114,400,114,400,106,415,96,...,1.360000,1.330000,0.0,1.270000,1.240000,1.210000,1.180000,1.8,"Hotham Hill, North Melbourne",3053
3,Carlton-Parkville,703,445,742,440,774,445,787,446,859,...,2.706667,2.636667,2.8,2.496667,2.426667,2.356667,2.286667,2.1,"Attwood, Westmeadows",3052
4,CBD-St Kilda Rd,"2,397",500,"2,371",505,"2,350",516,"2,351",520,"2,434",...,2.773333,2.653333,3.0,2.413333,2.293333,2.173333,2.053333,1.7,Scoresby,3182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Torquay,65,250,62,260,66,270,64,280,69,...,2.560000,2.580000,2.6,2.620000,2.640000,2.660000,2.680000,2.7,Ocean Grove,3228
140,Traralgon,134,190,153,190,153,190,167,190,173,...,2.233333,2.233333,1.9,2.233333,2.233333,2.233333,2.233333,2.4,Nicholson,3844
141,Warragul,77,200,77,205,89,210,89,210,91,...,2.546667,2.556667,2.8,2.576667,2.586667,2.596667,2.606667,2.5,"Valencia Creek, Briagolong, Coongulla, Maffra,...",3820
142,Warrnambool,331,230,341,230,366,230,367,235,401,...,2.773333,2.703333,2.6,2.563333,2.493333,2.423333,2.353333,2.3,"Bushfield, Grassmere, Winslow, Woodford",3280


In [70]:
test

{'Albert Park-Middle Park-West St Kilda':    Median_age_persons  Median_mortgage_repay_monthly  \
 0                37.6                         1647.7   
 1                37.7                         1640.6   
 2                37.8                         1633.6   
 3                37.9                         1626.5   
 
    Median_tot_prsnl_inc_weekly  Median_rent_weekly  Median_tot_fam_inc_weekly  \
 0                       1011.2               333.4                     2369.8   
 1                       1053.8               333.3                     2440.7   
 2                       1096.3               333.1                     2511.5   
 3                       1138.9               333.0                     2582.3   
 
    Average_num_psns_per_bedroom  Average_household_size  year  \
 0                           0.9                     1.8  2022   
 1                           0.9                     1.7  2023   
 2                           0.9                     1.6  2024

In [90]:
view = pd.DataFrame()
for i in range(0, 144):  
    view[bed_2_flat.loc[i, 'Suburb']] = test[bed_2_flat.loc[i, 'Suburb']]["Median_rental_price"]


view.to_csv("../data/curated/test_check.csv", index = False)

/tmp/ipykernel_1052/3441286843.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  view[bed_2_flat.loc[i, 'Suburb']] = test[bed_2_flat.loc[i, 'Suburb']]["Median_rental_price"]
/tmp/ipykernel_1052/3441286843.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  view[bed_2_flat.loc[i, 'Suburb']] = test[bed_2_flat.loc[i, 'Suburb']]["Median_rental_price"]
/tmp/ipykernel_1052/3441286843.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [55]:
print(bed_2_flat.loc[68, 'Suburb'])
test[bed_2_flat.loc[68, 'Suburb']]["Median_rental_price"]

East Brunswick


0    722.713212
1    722.352229
2    721.144682
3    720.885203
Name: Median_rental_price, dtype: float64